# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандидатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [57]:
import re
import textdistance
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from collections import Counter

In [58]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()
# создаем словарь
vocab = Counter(re.findall('\w+', corpus.lower()))

In [59]:
word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [60]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])

    # вся эффективноть берется из того, что мы сразу считаем близость
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее

    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn]

    return [(id2word[top], similarities[top]) for top in topn]

In [61]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    
    return max(closest)[0]

# N = sum(vocab.values())
#
# def P(word, N=N):
#     return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1


In [62]:
%time
get_closest_hybrid_match('крйг', X, vec)

CPU times: total: 0 ns
Wall time: 0 ns


'крэйг'

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [63]:
import re
from collections import Counter

In [64]:
class Symspell:
    def __init__(self, dictionary):
        self.dictionary = dictionary
        self.deletion_dict = self.build_deletion_dict()
        self.word_probabilities = self.build_probability_dict()

    def build_deletion_dict(self):
        deletion_dict = {}
        for word in self.dictionary:
            for i in range(len(word)):
                deletion = word[:i] + word[i+1:]
                if deletion not in deletion_dict:
                    deletion_dict[deletion] = []
                deletion_dict[deletion].append(word)
        return deletion_dict

    def build_probability_dict(self):
        word_counts = Counter(self.dictionary)
        total_words = sum(word_counts.values())
        probability_dict = {word: count / total_words for word, count in word_counts.items()}
        return probability_dict

    def correct(self, word):
        if word in self.dictionary:
            return word

        candidates = []
        for i in range(len(word)):
            deletion = word[:i] + word[i+1:]
            if deletion in self.deletion_dict:
                candidates.extend(self.deletion_dict[deletion])

        if not candidates:
            return word

        # выбираем кандидата с наибольшей вероятностью
        best_candidate = max(candidates, key=lambda candidate: self.word_probabilities.get(candidate, 0))

        return best_candidate

In [65]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()
# создаем словарь правильных слов
vocab = Counter(re.findall('\w+', corpus.lower()))

In [66]:
sym = Symspell(vocab)

In [67]:
%time
sym.correct('крйг')

CPU times: total: 0 ns
Wall time: 0 ns


'круг'

## *3. Чтение. (2 балла)

Прочитайте эту главу в книге Speech and Language Processing - https://web.stanford.edu/~jurafsky/slp3/2.pdf .
Ответьте на следующий вопрос:

1. Что такое Byte-Pair Encoding (опишите по-русски, как минимум 10 предложениями)?

*Это задание не связано напрямую с исправлением опечаток, но это важная тема, к которой мы вернемся в конце курса

Byte-Pair Encoding (BPE) - это алгоритм сжатия текстов, который впоследствии был использован OpenAI для токенизации при предварительном обучении модели GPT.
BPE отличается простотой и скоростью работы, что делает его подходящим для задач обработки естественного языка. Однако BPE имеет свои ограничения и недостатки.
Например, он может не всегда корректно обрабатывать редкие и неизвестные слова. Кроме того, качество работы BPE зависит от качества исходного словаря правильных слов.
Он используется многими моделями Transformer (включая GPT, GPT-2, RoBERTa, BART и DeBERTa).

BPE начинается с составления словаря правильных слов. На основе этого словаря создается словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово.
Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений.
Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления. Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово.

# Или

Byte-Pair Encoding (BPE) - это алгоритм сжатия текстов, который был предложен Sennrich и др. в 2016 году. Он начинается с составления словаря, который включает в себя все отдельные символы. Затем алгоритм анализирует обучающий корпус и выбирает два наиболее часто встречающихся рядом символа (например, ‘A’ и ‘B’), добавляет новый объединенный символ ‘AB’ в словарь и заменяет каждый смежный ‘A’ ‘B’ в корпусе на новый символ ‘AB’.

Этот процесс повторяется, пока не будет выполнено k объединений, создавая k новых токенов; k является параметром алгоритма. Результирующий словарь состоит из исходного набора символов плюс k новых символов.

Алгоритм обычно запускается внутри слов (не объединяя через границы слов), поэтому входной корпус сначала разделяется на пробелы, чтобы получить набор строк, каждая из которых соответствует символам слова, плюс специальный символ конца слова.

После обучения словаря используется сегментатор токенов для токенизации тестового предложения. Сегментатор токенов просто запускает на тестовых данных объединения, которые мы изучили из обучающих данных, жадно, в порядке, в котором мы их изучили.